In [1]:
import numpy as np
import uproot
from tqdm.notebook import tqdm
import os

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
# thinking about custom trajectory fitting, but decided it would be pretty complicated to distinguish
# between isochronous tracks and showers, which both have some spread around the real trajectory


In [2]:
# loading reco spacepoints from each event

reco_nu_spacepoints = {}
reco_all_cluster_spacepoints = {}
reco_nu_vtxs = {}
for file in tqdm(os.listdir("input_files/misclustering_candidate_nue_files")):
    if not file.endswith(".root"):
        continue

    f = uproot.open(f"input_files/misclustering_candidate_nue_files/{file}")

    rse_df = f["Trun"].arrays(["runNo", "subRunNo", "eventNo"], library="pd")

    run = int(rse_df["runNo"].iloc[0])
    subrun = int(rse_df["subRunNo"].iloc[0])
    event = int(rse_df["eventNo"].iloc[0])

    rse = (run, subrun, event)

    curr_reco_nu_spacepoints_dic = f["T_rec_charge_blob"].arrays(["x", "y", "z", "q"], library="np")
    curr_reco_nu_spacepoints = np.column_stack((curr_reco_nu_spacepoints_dic["x"], curr_reco_nu_spacepoints_dic["y"], curr_reco_nu_spacepoints_dic["z"], curr_reco_nu_spacepoints_dic["q"]))

    curr_all_cluster_spacepoints_dic = f["T_cluster"].arrays(["x", "y", "z", "q"], library="np")
    curr_all_cluster_spacepoints = np.column_stack((curr_all_cluster_spacepoints_dic["x"], curr_all_cluster_spacepoints_dic["y"], curr_all_cluster_spacepoints_dic["z"], curr_all_cluster_spacepoints_dic["q"]))

    reco_nu_spacepoints[rse] = curr_reco_nu_spacepoints
    reco_all_cluster_spacepoints[rse] = curr_all_cluster_spacepoints

    reco_nu_vtx_dic = f["T_vtx"].arrays(["x", "y", "z", "flag_main"], library="np")
    reco_nu_vtx_x = reco_nu_vtx_dic["x"][reco_nu_vtx_dic["flag_main"] == 1][0]
    reco_nu_vtx_y = reco_nu_vtx_dic["y"][reco_nu_vtx_dic["flag_main"] == 1][0]
    reco_nu_vtx_z = reco_nu_vtx_dic["z"][reco_nu_vtx_dic["flag_main"] == 1][0]

    reco_nu_vtxs[rse] = (reco_nu_vtx_x, reco_nu_vtx_y, reco_nu_vtx_z)


  0%|          | 0/85 [00:00<?, ?it/s]

In [3]:
# making detector boundary points

tpc_min_x = -1.
tpc_max_x = 254.3
tpc_min_y = -115.
tpc_max_y = 117.
tpc_min_z = 0.6
tpc_max_z = 1036.4

def generate_box_edge_points(x_min, x_max, y_min, y_max, z_min, z_max, num_points_per_edge=10):

    # Generate points along edges parallel to X-axis
    t = np.linspace(0, 1, num_points_per_edge)
    x_edges = [
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_min),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_max),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_min),
            np.full_like(t, z_max)
        ]),
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_max),
            np.full_like(t, z_max)
        ])
    ]
    
    # Generate points along edges parallel to Y-axis
    y_edges = [
        np.column_stack([
            np.full_like(t, x_min),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_min),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_max)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_max)
        ])
    ]
    
    # Generate points along edges parallel to Z-axis
    z_edges = [
        np.column_stack([
            np.full_like(t, x_min),
            np.full_like(t, y_min),
            z_min + t * (z_max - z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            np.full_like(t, y_min),
            z_min + t * (z_max - z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_min),
            np.full_like(t, y_max),
            z_min + t * (z_max - z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            np.full_like(t, y_max),
            z_min + t * (z_max - z_min)
        ])
    ]
    
    # Combine all edges
    all_points = np.vstack(x_edges + y_edges + z_edges)
    return all_points


detector_boundary_points = generate_box_edge_points(tpc_min_x, tpc_max_x, tpc_min_y, tpc_max_y, tpc_min_z, tpc_max_z, num_points_per_edge=100)
x_width = tpc_max_x - tpc_min_x
expanded_detector_boundary_points = generate_box_edge_points(tpc_min_x - x_width, tpc_max_x + x_width, tpc_min_y, tpc_max_y, tpc_min_z, tpc_max_z, num_points_per_edge=100)


In [4]:
rse_list = list(reco_nu_spacepoints.keys())

index = 50

# 10 is good
# 50 is perfect
# 51 is good
# 54 is good
# 55 is okay
# 57 is good, with a true cosmic reco nu track

rse = rse_list[index]

print("rse:", rse)

curr_detector_boundary_points = detector_boundary_points
curr_expanded_detector_boundary_points = expanded_detector_boundary_points

curr_reco_all_cluster_spacepoints = reco_all_cluster_spacepoints[rse]

curr_reco_nu_spacepoints = reco_nu_spacepoints[rse]

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])

fig.add_trace(go.Scatter3d(
    x=curr_expanded_detector_boundary_points[:, 2],
    y=curr_expanded_detector_boundary_points[:, 0],
    z=curr_expanded_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=0.2,
        color='black',
        opacity=0.8
    ),
    name='Expanded TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=curr_detector_boundary_points[:, 2],
    y=curr_detector_boundary_points[:, 0],
    z=curr_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='black',
        opacity=0.8
    ),
    name='TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=[reco_nu_vtxs[rse][2]],
    y=[reco_nu_vtxs[rse][0]],
    z=[reco_nu_vtxs[rse][1]],
    mode='markers',
    marker=dict(size=10, color='orange', opacity=1),
    name='Reco Neutrino Vertex'
))

fig.add_trace(go.Scatter3d(
    x=curr_reco_all_cluster_spacepoints[:, 2],
    y=curr_reco_all_cluster_spacepoints[:, 0],
    z=curr_reco_all_cluster_spacepoints[:, 1],
    mode='markers',
    marker=dict(size=1, color='blue', opacity=0.8),
    name='Reco All Cluster Spacepoints'
))

fig.update_layout(
    scene=dict(
        xaxis_title='z',
        yaxis_title='x',
        zaxis_title='y',
        aspectratio=dict(
            x=5,
            y=3,
            z=1
        ),
    ),
    width=2000,
    height=1200,
    autosize=False,
    scene_camera=dict(
        eye=dict(x=-1.5, y=-1.5, z=1.5)
    )
)

fig.show(renderer="browser")


rse: (13779, 223, 11160)
